In [1]:
## imports
import pandas as pd
import numpy as np
import mne
import re
import os

from mne.preprocessing import (
    ICA, 
    create_ecg_epochs, 
    corrmap, 
    create_eog_epochs
)

# mne graphics config
# from PySide6 import QtCore, QtGui, QtWidgets, __version__ 

rawa = None
rawb = None

import datetime
print('IMPORTED MODULES today:', datetime.datetime.now())

# Removed cached function import
import sys
try:
    del sys.modules['rename_events']
except KeyError:
    pass

from rename_events import relabel_events


# setup

# SESSION AND POSNER CODES
session_id='m_00_02'
posner_id ='pos2a'
posner_id_b ='pos2b'

IMPORTED MODULES today: 2023-07-17 14:21:57.095043


In [6]:
%matplotlib qt

# Posner A

In [2]:
# Relabel stim events and load raw data
raw_a = relabel_events(session_id, posner_id)

Data file:  m_00_02/m_00_02_pos2a.vhdr


/Users/babe/src/gla/dissertation/go/ica_notebooks/rename_events.py:50: RuntimeWarning: No coordinate information found for channels ['ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  raw = mne.io.read_raw_brainvision(vhdr_file, preload=True, verbose=0)


Used Annotations descriptions: ['New Segment/', 'Stimulus/S 10', 'Stimulus/S 30', 'Stimulus/S 40', 'Stimulus/S 50', 'Stimulus/S 51', 'Stimulus/S 60', 'Stimulus/S 70', 'Stimulus/S 80', 'Stimulus/S101', 'Stimulus/S109']
Found '2a' in file: m_00_02/00_2a_posner_task_2023-04-06_14h21.27.407/00_2a_posner_task_2023-04-06_14h21.27.407.csv
Found '2b' in file: m_00_02/00_2b_posner_task_2023-04-06_15h22.44.265/00_2b_posner_task_2023-04-06_15h22.44.265.csv
********** 2a
Used Annotations descriptions: ['New Segment/', 'Stimulus/S 10', 'Stimulus/S 30', 'Stimulus/S 40', 'Stimulus/S 50', 'Stimulus/S 51', 'Stimulus/S 60', 'Stimulus/S 70', 'Stimulus/S 80', 'Stimulus/S101', 'Stimulus/S109']
Cropping annotations 2023-04-06 14:21:06.377167+00:00 - 2023-04-06 14:36:00.357167+00:00
  [0] Keeping  (2023-04-06 14:21:06.377167+00:00 - 2023-04-06 14:21:06.377167+00:00 -> 0.0 - 0.0)
  [1] Keeping  (2023-04-06 14:23:17.968167+00:00 - 2023-04-06 14:23:17.968167+00:00 -> 131.591 - 131.591)
  [2] Keeping  (2023-04-0

/Users/babe/src/gla/dissertation/go/ica_notebooks/rename_events.py:50: RuntimeWarning: Not setting position of 1 misc channel found in montage:
['ECG']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  raw = mne.io.read_raw_brainvision(vhdr_file, preload=True, verbose=0)


  [400] Keeping  (2023-04-06 14:34:26.395167+00:00 - 2023-04-06 14:34:26.395167+00:00 -> 800.018 - 800.018)
  [401] Keeping  (2023-04-06 14:34:29.026167+00:00 - 2023-04-06 14:34:29.026167+00:00 -> 802.649 - 802.649)
  [402] Keeping  (2023-04-06 14:34:29.126167+00:00 - 2023-04-06 14:34:29.126167+00:00 -> 802.749 - 802.749)
  [403] Keeping  (2023-04-06 14:34:31.396167+00:00 - 2023-04-06 14:34:31.396167+00:00 -> 805.019 - 805.019)
  [404] Keeping  (2023-04-06 14:34:32.896167+00:00 - 2023-04-06 14:34:32.896167+00:00 -> 806.519 - 806.519)
  [405] Keeping  (2023-04-06 14:34:35.636167+00:00 - 2023-04-06 14:34:35.636167+00:00 -> 809.259 - 809.259)
  [406] Keeping  (2023-04-06 14:34:35.736167+00:00 - 2023-04-06 14:34:35.736167+00:00 -> 809.359 - 809.359)
  [407] Keeping  (2023-04-06 14:34:37.896167+00:00 - 2023-04-06 14:34:37.896167+00:00 -> 811.519 - 811.519)
  [408] Keeping  (2023-04-06 14:34:39.396167+00:00 - 2023-04-06 14:34:39.396167+00:00 -> 813.019 - 813.019)
  [409] Keeping  (2023-04-06

In [3]:
event_map = {'1': 11, '10': 10, '101': 101, '109': 109, '11': 11, '117': 117, '20': 20, '30': 30, '40': 40, '5': 5, '50': 50, '51': 51, '52': 52, '60': 60, '70': 70, '80': 80, '99999': 99999}

# get events and times
events, event_id = mne.events_from_annotations(raw_a, event_map)

e_a = events
# all rows with fixation (code 40)
e_40_a = e_a[e_a[:, 2] == 40]
# all rows with trial end (code 101)
e_101_a = e_a[e_a[:,2]==101]

a_tmin= e_40_a[0][0]/1000
a_tmax= e_101_a[-1][0]/1000

# Crop and make a copy and filter
filt_raw_a = raw_a.crop(tmin=a_tmin, tmax=a_tmax).copy().filter(l_freq=1.0, h_freq=40)

epochs_a = mne.Epochs(filt_raw_a, 
                    e_a, 
                    event_id=None, 
                    tmin=-0.2, 
                    tmax=0.5, 
                    preload=True, 
                    baseline=(None, 0))

Used Annotations descriptions: ['10', '101', '109', '30', '40', '50', '51', '52', '60', '70', '80', '99999']
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (3.301 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Not setting metadata
413 matching events found
Setting baseline interval to [-0.2, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 413 events and 701 original time points ...
4 bad epochs dropped


[Parallel(n_jobs=1)]: Done  63 out of  63 | elapsed:    0.7s finished


In [50]:
raw_a.plot()

Using pyopengl with version 3.1.6


Channels marked as bad:
none
Channels marked as bad:
none


In [4]:
# Fitting ICA
ica = ICA(n_components=len(filt_raw_a.ch_names)-1, max_iter="auto", random_state=999)
ica.fit(filt_raw_a)
ica

Fitting ICA to data using 63 channels (please be patient, this may take a while)
Selecting by number: 63 components
Fitting ICA took 38.1s.


Method,fastica
Fit,52 iterations on raw data (675589 samples)
ICA components,63
Available PCA components,63
Channel types,eeg
ICA components marked for exclusion,—


In [7]:
ica.plot_components()

[<MNEFigure size 975x967 with 20 Axes>,
 <MNEFigure size 975x967 with 20 Axes>,
 <MNEFigure size 975x967 with 20 Axes>,
 <MNEFigure size 585x260 with 3 Axes>]

In [ ]:
raw_a.load_data()
ica.plot_sources(raw_a)

Creating RawArray with float64 data, n_channels=63, n_times=675589
    Range : 144915 ... 820503 =    144.915 ...   820.503 secs
Ready.
Using qt as 2D backend.
Using pyopengl with version 3.1.6


    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
337 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
337 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
337 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
337 matching events found
No baseline correction applied
0 projection items activated


In [8]:
print(ica.exclude)

[41, 53, 59, 21, 20, 0, 1, 2, 3, 8, 9, 11]


In [9]:
reconstr_raw = raw_a.copy()
# Apply ica to copy
ica.apply(reconstr_raw)
# Plot
raw_a.plot(title="Raw data")
reconstr_raw.plot(title="ICA applied")

Applying ICA to Raw instance
    Transforming to ICA space (63 components)
    Zeroing out 12 ICA components
    Projecting back using 63 PCA components
Using qt as 2D backend.
Using pyopengl with version 3.1.6
Using pyopengl with version 3.1.6


Channels marked as bad:
none
Channels marked as bad:
none


In [10]:
# Save file
file_name = session_id+'_'+posner_id+'.vhdr'
output_dir='cleaned'

print(file_name)
cur_dir = os.getcwd()
session_dir = os.path.join(cur_dir, output_dir, session_id)
print(session_dir)

if not os.path.exists(session_dir):
   os.makedirs(session_dir)

print("\n**** Saving file to: ", session_dir+'/'+file_name) 
mne.export.export_raw(session_dir+'/'+file_name, reconstr_raw, verbose=1)

m_00_02_pos2a.vhdr
/Users/babe/src/gla/dissertation/go/ica_notebooks/cleaned/m_00_02

**** Saving file to:  /Users/babe/src/gla/dissertation/go/ica_notebooks/cleaned/m_00_02/m_00_02_pos2a.vhdr


/Applications/MNE-Python/.mne-python/lib/python3.10/site-packages/pybv/_export.py:61: RuntimeWarning: Encountered data in 'short' format. Converting to float32.
  warn(
/Applications/MNE-Python/.mne-python/lib/python3.10/site-packages/pybv/io.py:690: UserWarning: Encountered unsupported non-voltage units: n/a
Note that the BrainVision format specification supports only µV.
  warn(msg)


In [11]:
!tree {session_dir}

/Users/babe/src/gla/dissertation/go/ica_notebooks/cleaned/m_00_02
├── m_00_02_pos2a.eeg
├── m_00_02_pos2a.vhdr
└── m_00_02_pos2a.vmrk

1 directory, 3 files


In [12]:
del raw_a
del reconstr_raw
del ica
del filt_raw_a

In [52]:
### TESTING ###
# test = mne.io.read_raw_brainvision(session_dir+'/'+file_name, preload=True)

In [53]:
# test.plot()

# Posner B

In [13]:
# Relabel stim events and load raw data
raw_b = relabel_events(session_id, posner_id_b)

Data file:  m_00_02/m_00_02_pos2b.vhdr


/Users/babe/src/gla/dissertation/go/ica_notebooks/rename_events.py:50: RuntimeWarning: No coordinate information found for channels ['ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  raw = mne.io.read_raw_brainvision(vhdr_file, preload=True, verbose=0)


Used Annotations descriptions: ['New Segment/', 'Stimulus/S 10', 'Stimulus/S 20', 'Stimulus/S 30', 'Stimulus/S 40', 'Stimulus/S 50', 'Stimulus/S 51', 'Stimulus/S 60', 'Stimulus/S 70', 'Stimulus/S 80', 'Stimulus/S101', 'Stimulus/S109', 'Stimulus/S117']
Found '2a' in file: m_00_02/00_2a_posner_task_2023-04-06_14h21.27.407/00_2a_posner_task_2023-04-06_14h21.27.407.csv
Found '2b' in file: m_00_02/00_2b_posner_task_2023-04-06_15h22.44.265/00_2b_posner_task_2023-04-06_15h22.44.265.csv
********** 2b
Used Annotations descriptions: ['New Segment/', 'Stimulus/S 10', 'Stimulus/S 20', 'Stimulus/S 30', 'Stimulus/S 40', 'Stimulus/S 50', 'Stimulus/S 51', 'Stimulus/S 60', 'Stimulus/S 70', 'Stimulus/S 80', 'Stimulus/S101', 'Stimulus/S109', 'Stimulus/S117']
Cropping annotations 2023-04-06 15:24:13.788051+00:00 - 2023-04-06 15:37:43.508051+00:00
  [0] Keeping  (2023-04-06 15:24:13.788051+00:00 - 2023-04-06 15:24:13.788051+00:00 -> 0.0 - 0.0)
  [1] Keeping  (2023-04-06 15:25:00.656051+00:00 - 2023-04-06 1

/Users/babe/src/gla/dissertation/go/ica_notebooks/rename_events.py:50: RuntimeWarning: Not setting position of 1 misc channel found in montage:
['ECG']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  raw = mne.io.read_raw_brainvision(vhdr_file, preload=True, verbose=0)


  [304] Keeping  (2023-04-06 15:34:25.288051+00:00 - 2023-04-06 15:34:25.288051+00:00 -> 611.5 - 611.5)
  [305] Keeping  (2023-04-06 15:34:26.788051+00:00 - 2023-04-06 15:34:26.788051+00:00 -> 613.0 - 613.0)
  [306] Keeping  (2023-04-06 15:34:29.558051+00:00 - 2023-04-06 15:34:29.558051+00:00 -> 615.77 - 615.77)
  [307] Keeping  (2023-04-06 15:34:29.658051+00:00 - 2023-04-06 15:34:29.658051+00:00 -> 615.87 - 615.87)
  [308] Keeping  (2023-04-06 15:34:31.791051+00:00 - 2023-04-06 15:34:31.791051+00:00 -> 618.003 - 618.003)
  [309] Keeping  (2023-04-06 15:34:38.533051+00:00 - 2023-04-06 15:34:38.533051+00:00 -> 624.745 - 624.745)
  [310] Keeping  (2023-04-06 15:34:40.627051+00:00 - 2023-04-06 15:34:40.627051+00:00 -> 626.839 - 626.839)
  [311] Keeping  (2023-04-06 15:34:40.628051+00:00 - 2023-04-06 15:34:40.628051+00:00 -> 626.84 - 626.84)
  [312] Keeping  (2023-04-06 15:34:42.128051+00:00 - 2023-04-06 15:34:42.128051+00:00 -> 628.34 - 628.34)
  [313] Keeping  (2023-04-06 15:34:45.288051

In [14]:
event_map = {'1': 11, '10': 10, '101': 101, '109': 109, '11': 11, '117': 117, '20': 20, '30': 30, '40': 40, '5': 5, '50': 50, '51': 51, '52': 52, '60': 60, '70': 70, '80': 80, '99999': 99999}

# get events and times
events_b, event_id_b = mne.events_from_annotations(raw_b, event_map)

e_b = events_b
# all rows with fixation (code 40)
e_40_b = e_b[e_b[:, 2] == 40]
# all rows with trial end (code 101)
e_101_b = e_b[e_b[:,2]==101]

b_tmin= e_40_b[0][0]/1000
b_tmax= e_101_b[-1][0]/1000

# Crop and make a copy and filter
filt_raw_b = raw_b.crop(tmin=b_tmin, tmax=b_tmax).copy().filter(l_freq=1.0, h_freq=40)

epochs_b = mne.Epochs(filt_raw_b, 
                    e_b, 
                    event_id=None, 
                    tmin=-0.2, 
                    tmax=0.5, 
                    preload=True, 
                    baseline=(None, 0))

Used Annotations descriptions: ['10', '101', '109', '117', '20', '30', '40', '50', '51', '52', '60', '70', '80', '99999']
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (3.301 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Not setting metadata
415 matching events found
Setting baseline interval to [-0.2, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 415 events and 701 original time points ...
6 bad epochs dropped


[Parallel(n_jobs=1)]: Done  63 out of  63 | elapsed:    0.6s finished


In [5]:
raw_b.plot()

Using qt as 2D backend.
Using pyopengl with version 3.1.6


Channels marked as bad:
none


In [15]:
# Fitting ICA
ica_b = ICA(n_components=len(filt_raw_b.ch_names)-1, max_iter="auto", random_state=999)
ica_b.fit(filt_raw_b, verbose='DEBUG')
ica_b

Fitting ICA to data using 63 channels (please be patient, this may take a while)
Selecting by number: 63 components
Cropping annotations 2023-04-06 15:26:05.485051+00:00 - 2023-04-06 15:37:25.175051+00:00
  [0] Keeping  (2023-04-06 15:26:05.485051+00:00 - 2023-04-06 15:26:05.485051+00:00 -> 111.697 - 111.697)
  [1] Keeping  (2023-04-06 15:26:06.985051+00:00 - 2023-04-06 15:26:06.985051+00:00 -> 113.197 - 113.197)
  [2] Keeping  (2023-04-06 15:26:10.615051+00:00 - 2023-04-06 15:26:10.615051+00:00 -> 116.827 - 116.827)
  [3] Keeping  (2023-04-06 15:26:10.715051+00:00 - 2023-04-06 15:26:10.715051+00:00 -> 116.927 - 116.927)
  [4] Keeping  (2023-04-06 15:26:11.995051+00:00 - 2023-04-06 15:26:11.995051+00:00 -> 118.207 - 118.207)
  [5] Keeping  (2023-04-06 15:26:13.495051+00:00 - 2023-04-06 15:26:13.495051+00:00 -> 119.707 - 119.707)
  [6] Keeping  (2023-04-06 15:26:16.415051+00:00 - 2023-04-06 15:26:16.415051+00:00 -> 122.627 - 122.627)
  [7] Keeping  (2023-04-06 15:26:16.515051+00:00 - 20

Method,fastica
Fit,126 iterations on raw data (679690 samples)
ICA components,63
Available PCA components,63
Channel types,eeg
ICA components marked for exclusion,—


In [16]:
ica_b.plot_components()

[<MNEFigure size 975x967 with 20 Axes>,
 <MNEFigure size 975x967 with 20 Axes>,
 <MNEFigure size 975x967 with 20 Axes>,
 <MNEFigure size 585x260 with 3 Axes>]

In [ ]:
raw_b.load_data()
ica_b.plot_sources(raw_b)

Creating RawArray with float64 data, n_channels=63, n_times=679690
    Range : 111697 ... 791386 =    111.697 ...   791.386 secs
Ready.
Using pyopengl with version 3.1.6


    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
339 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
339 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
339 matching events found
No baseline correction applied
0 projection items activated


In [17]:
ica_b.exclude

[25, 26, 23, 21, 0, 1, 2, 3, 4, 9, 8, 17]

In [18]:
reconstr_raw_b = raw_b.copy()
# Apply ica to copy
ica_b.apply(reconstr_raw_b)
# Plot and compare
raw_b.plot(title="Raw data (posner b)")
reconstr_raw_b.plot(title="ICA applied (posner b)")

Applying ICA to Raw instance
    Transforming to ICA space (63 components)
    Zeroing out 12 ICA components
    Projecting back using 63 PCA components
Using pyopengl with version 3.1.6
Using pyopengl with version 3.1.6


Channels marked as bad:
none
Channels marked as bad:
none


In [19]:
# Save file
file_name_b = session_id+'_'+posner_id_b+'.vhdr'
output_dir_b='cleaned'
cur_dir = os.getcwd()
session_dir = os.path.join(cur_dir, output_dir_b, session_id)
if not os.path.exists(session_dir):
   os.makedirs(session_dir)

print("\n**** Saving file to: ", session_dir+'/'+file_name_b) 
mne.export.export_raw(session_dir+'/'+file_name_b, reconstr_raw_b, verbose=1)


**** Saving file to:  /Users/babe/src/gla/dissertation/go/ica_notebooks/cleaned/m_00_02/m_00_02_pos2b.vhdr


/Applications/MNE-Python/.mne-python/lib/python3.10/site-packages/pybv/_export.py:61: RuntimeWarning: Encountered data in 'short' format. Converting to float32.
  warn(
/Applications/MNE-Python/.mne-python/lib/python3.10/site-packages/pybv/io.py:690: UserWarning: Encountered unsupported non-voltage units: n/a
Note that the BrainVision format specification supports only µV.
  warn(msg)


In [20]:
!tree {session_dir}

/Users/babe/src/gla/dissertation/go/ica_notebooks/cleaned/m_00_02
├── m_00_02_pos2a.eeg
├── m_00_02_pos2a.vhdr
├── m_00_02_pos2a.vmrk
├── m_00_02_pos2b.eeg
├── m_00_02_pos2b.vhdr
└── m_00_02_pos2b.vmrk

1 directory, 6 files


# ...

In [21]:
del raw_b
del reconstr_raw_b
del ica_b
del filt_raw_b